In [ ]:
import jax
jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as np 
from matplotlib import pyplot as plt 

import s2scat 
import s2fft
import s2wav

In [ ]:
L = 1024
N = 3
reality = True
f = np.load("texture_maps/venus_1024.npy")
flm = np.load("texture_maps/venus_alm_1024.npy")

In [ ]:
plt.imshow(f, cmap='twilight')
plt.show()

In [ ]:
wavelets = s2wav.filters.filters_directional_vectorised(L, N)[0]
matrices = s2scat.operators.matrices.generate_recursive_matrices(L, N, reality=reality)
norm = s2scat.utility.normalisation.compute_norm(flm, L, N, reality=reality, filters=wavelets, precomps=matrices, recursive=True)

In [ ]:
target_coefficients = s2scat.core.scatter.directional(
    flm, 
    L, 
    N, 
    reality=reality,
    filters=wavelets,
    norm=norm,
    precomps=matrices,
    recursive=True
)